In [1]:
# Load aadc library

import aadc
import numpy as np

bind_swig2pybind_linearalgebra_i
Adding extend interface from SWIG to Pybind11 for class FixedLocalVolSurface
Adding extend interface from SWIG to Pybind11 for class GridModelLocalVolSurface


In [2]:
# Regular python lists
Dates = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Rates = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]
print(type(Rates))

<class 'list'>


In [3]:
# Python list with mixed type elements
iRates = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, aadc.idouble(0.09), aadc.idouble(0.1)]
print(type(iRates))

<class 'list'>


In [4]:
# Python list with all elements of type aadc.idouble
iiRates = [aadc.idouble(0.01), aadc.idouble(0.02), aadc.idouble(0.03), aadc.idouble(0.04), aadc.idouble(0.05), aadc.idouble(0.06), aadc.idouble(0.07), aadc.idouble(0.08), aadc.idouble(0.09), aadc.idouble(0.1)]

In [5]:
# Numpy arrays type
nRates = np.ones(10)
print(np.log(iRates[8]))

-2.407945608651872


# Here MyDemoCurve is defined on C++ side as follows:
class MyDemoCurve {
public:
    MyDemoCurve(const std::vector<idouble> &d, const std::vector<idouble> &r) : d_(d), r_(r) {
     PYAADC_DEBUG std::cout << "Constructing MyCurve\n";

    }
    idouble operator()(const idouble& x) const {
        return x * (r_[0] + r_.back());
    }
private:
     std::vector<idouble> d_;
     std::vector<idouble> r_;
};

In [6]:
# Create a curve with a python list of native doubles and numpy array of native doubles
forecast_curve = aadc.MyDemoCurve(Dates, nRates)
nRates[9] = aadc.idouble(0.1)       # automatically converts to double and this is outside of recording scope
forecast_curve = aadc.MyDemoCurve(Dates, nRates)

In [7]:
# Create curves with different types of rates
forecast_curve_n = aadc.MyDemoCurve(Dates, Rates)
forecast_curve_ii = aadc.MyDemoCurve(Dates, iiRates)
forecast_curve = aadc.MyDemoCurve(Dates, iRates)

In [8]:
# Outside of recording scope all objects return native doubles
print (forecast_curve_n(1))
print (forecast_curve_ii(1))
print (forecast_curve(1))

0.11
0.11
0.11


In [10]:
# Lets see what happens when we start recording
print ("Start recording")
funcs = aadc.Functions()
funcs.start_recording()

Start recording
You are using evaluation version of AADC. Expire date is 20240901


In [11]:
# We mark one idouble as input iRates[8] is idouble but not marked as input
arg1 = iRates[9].mark_as_input()
arg1

Arg(6)

In [12]:
# Recreate curve with arrays
forecast_curve = aadc.MyDemoCurve(Dates, iRates)
# passive curve only gets native doubles as input
forecast_curve_passive = aadc.MyDemoCurve(Dates, Rates)

In [13]:
# returns active double because internaly iRates[9] 
# is used with the constant 1.0 argument
forecast_curve(1)

idouble([AAD[rv] [adj] :9,1.10e-01])

In [14]:
# Returns native double because all inputs are native doubles
# and the curve internally uses native doubles
forecast_curve_passive(1)

0.11

In [15]:
# Returns native double because all input is idouble, but
# not marked as input and the curve internally uses native doubles
forecast_curve_passive(aadc.idouble(1))

0.11

In [16]:
# Returns active idouble because argument is marked as input
# Note that internally the curve uses native doubles
forecast_curve_passive(iRates[9])

idouble([AAD[rv] [adj] :7,1.10e-02])

In [17]:
# Simple expression with marked idoubles return active idouble
exp1 = iRates[9] + iRates[9]
exp1

idouble([AAD[rv] [adj] :10,2.00e-01])

In [18]:
# Simple expression with unmarked idoubles return native double
iRates[8] + iRates[8]

0.18

In [19]:
# Simple expression with unmarked idoubles can be converted to float
float(iRates[8] + iRates[8])

0.18

In [20]:
# During recording, converting active idouble to float returns exception.
try:
    print(float(exp1)) # returns exception
except Exception as e:
    print (e)


idouble->double conversion is found at   File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/dimach/venv/test/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()

  File "/home/dimach/venv/test/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()

  File "/home/dimach/venv/test/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()

  File "/home/dimach/venv/test/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()

  File "/usr/lib/python3.11/asyncio/base_events.py", line 607, in run_forever
    self._run_once()

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once
    handle._run()

  File "/usr/lib/python3.11/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *s

In [21]:
# NumPy scalar operations with idoubles
np.log(exp1)

idouble([AAD[rv] [adj] :14,-1.61e+00])

In [22]:
funcs.stop_recording()
print ("Stop recording")

Stop recording


In [23]:
# exp1 is still idouble but now it safe to convert to float
print(type(exp1))
np.log(exp1)

<class 'aadc._aadc_core.idouble'>


-1.6094379124341003

In [24]:
# The curve was created during recording but now it returns native double
forecast_curve(1)

0.11